# 

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

import pandas as pd
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC



Cake & Tarts

Step 1: Click, navigate, and get all the product urls

In [2]:
# Initialize the driver with headless mode
options = Options()
options.headless = True
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Navigate to the initial web page
driver.get('https://www.sainsburys.co.uk/gol-ui/groceries/bakery/cakes-and-tarts/all-cakes-and-tarts/c:1018792')
time.sleep(2)  # Allow the page to load

# Function to click with JavaScript
def click_with_javascript(element):
    driver.execute_script("arguments[0].click();", element)

# Handle cookie consent
try:
    consent_button = driver.find_element(By.ID, "onetrust-accept-btn-handler")
    consent_button.click()
    time.sleep(1)  # Allow time for overlays to disappear
except NoSuchElementException:
    print("Consent button not found.")

# Initialize a master list to hold all product links
master_product_links = []

# Function to navigate pages and extract product links
def navigate_pages():
    all_product_links = []
    while True:
        # Wait for the product links to be loaded and extract them
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a.pt__link')))
        soup = BeautifulSoup(driver.page_source, "html.parser")
        product_links = [a['href'] for a in soup.find_all('a', class_="pt__link")]
        product_links = list(dict.fromkeys(product_links))  # Remove duplicates
        all_product_links.extend(product_links)

        try:
            next_page_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.ln-c-pagination__link[aria-label="Next page"]')))
            next_page_button.click()
            print("Next page button clicked.")
            time.sleep(2)
        except Exception:
            print("No more next page buttons.")
            break
    
    return all_product_links

# Click pill buttons, navigate pages, and extract product links
for index, button in enumerate(driver.find_elements(By.CLASS_NAME, 'browse-header-pill')):
    try:
        click_with_javascript(button)
        print(f"Pill button {index + 1} clicked.")
        product_links = navigate_pages()  # Navigate and extract links
        master_product_links.extend(product_links)  # Add to the master list
        print(f"Extracted {len(product_links)} product links.")
    except Exception as e:
        print(f"Error clicking pill button {index + 1}: {str(e)}")
        break

# Close the browser
driver.quit()

# Now you can print, save, or process the master list of product links
print(f"Total extracted product links: {len(master_product_links)}")
# Optionally, save the list to a file or database, or process it as needed


Pill button 1 clicked.
Next page button clicked.
Next page button clicked.
Next page button clicked.
No more next page buttons.
Extracted 211 product links.
Pill button 2 clicked.
No more next page buttons.
Extracted 27 product links.
Pill button 3 clicked.
Next page button clicked.
No more next page buttons.
Extracted 75 product links.
Pill button 4 clicked.
No more next page buttons.
Extracted 5 product links.
Pill button 5 clicked.
No more next page buttons.
Extracted 16 product links.
Pill button 6 clicked.
No more next page buttons.
Extracted 41 product links.
Pill button 7 clicked.
No more next page buttons.
Extracted 58 product links.
Pill button 8 clicked.
No more next page buttons.
Extracted 49 product links.
Total extracted product links: 482


In [3]:
# Remove dupilication
mylist = list(dict.fromkeys(master_product_links))
print(len(mylist))

223


Nutrition

In [4]:
all_nutrition_data = []

for url in mylist:
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(url)
    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'h1')))


    page_source = driver.page_source
    soup = BeautifulSoup(page_source, "html.parser")

    title = soup.find('h1', class_='pd__header')
    title_text = title.text.strip() if title else 'Title Not Found'

# Description ver 1
    description_text = 'Description Not Found'
    
    description_header_v1 = soup.find('h3', class_='productDataItemHeader', string='Description')
    if description_header_v1:
        product_text_div4 = description_header_v1.find_next_sibling('div', class_='productText')
        if product_text_div4:
        # Find the first <p> tag that contains actual text. This assumes the ingredients are in the first <p> tag with content.
            p_tags4 = product_text_div4.find_all('p')
            for p_tag4 in p_tags4:
                if p_tag4.get_text(strip=True):  # Check if the paragraph has text
                    description_text = p_tag4.get_text(strip=True)
                    
    # Description version 2
    description_header_v2 = soup.find('h3', string='Description')
    if description_header_v2:
        product_text_div5 = description_header_v2.find_next_sibling('div', class_='memo')
        if product_text_div5:
        # Find the first <p> tag that contains actual text. This assumes the ingredients are in the first <p> tag with content.
            p_tags5 = product_text_div5.find_all('p')
            for p_tag5 in p_tags5:
                if p_tag5.get_text(strip=True):  # Check if the paragraph has text
                    description_text = p_tag5.get_text(strip=True)
                    break  # Stop at the first <p> tag with content


    # Description version 3
    description_header_v3 = soup.find('h3', string='Description')
    if description_header_v3:
        product_text_div6 = description_header_v3.find_next_sibling('div', class_='longTextItems')
        if product_text_div6:
        # Find the first <p> tag that contains actual text. This assumes the ingredients are in the first <p> tag with content.
            p_tags6 = product_text_div6.find_all('p')
            for p_tag6 in p_tags6:
                if p_tag6.get_text(strip=True):  # Check if the paragraph has text
                    description_text = p_tag6.get_text(strip=True)
                    break  # Stop at the first <p> tag with content
            
        # For example, replacing '&nbsp;' with a space, and '&amp;' with '&'
            description_text = description_text.replace('&nbsp;', ' ').replace('&amp;', '&')    

    price_span = soup.find('span', class_='pd__cost__retail-price')
    price_text = price_span.text.strip() if price_span else 'Price Not Found'

    # Initialize ingredients_text
    ingredients_text = 'Ingredients Not Found'

    # Checking for Version 1 structure
    ingredients_header_v1 = soup.find('h3', class_='itemHeader', string='Ingredients')
    if ingredients_header_v1:
        ul = ingredients_header_v1.find_next_sibling('ul', class_='productIngredients')
        if ul:
            ingredients = [li.get_text(strip=True) for li in ul.find_all('li')]
            ingredients_text = ', '.join(ingredients).replace(',,', ', ')

    # Version 2 structure: Direct search for INGREDIENTS text within <div class="productText">
    if ingredients_text == 'Ingredients Not Found':
        divs = soup.find_all('div', class_='productText')
        for div in divs:
            p_tags = div.find_all('p')
            for p_tag in p_tags:
                if 'INGREDIENTS:' in p_tag.get_text():
                    ingredients_text = p_tag.get_text(separator=" ", strip=True).replace('INGREDIENTS:', '').strip()
                    break
            if ingredients_text != 'Ingredients Not Found':
                break
    # Version 3
    ingredients_header_v2 = soup.find('h3', string='Ingredients')
    if ingredients_header_v2:
        product_text_div1 = ingredients_header_v2.find_next_sibling('div', class_='productText')
        if product_text_div1:
        # Find the first <p> tag that contains actual text. This assumes the ingredients are in the first <p> tag with content.
            p_tags1 = product_text_div1.find_all('p')
            for p_tag1 in p_tags1:
                if p_tag1.get_text(strip=True):  # Check if the paragraph has text
                    ingredients_text = p_tag1.get_text(strip=True)
                    break  # Stop at the first <p> tag with content

        # Optionally, you might want to clean up the ingredients_text here
        # For example, replacing '&nbsp;' with a space, and '&amp;' with '&'
            ingredients_text = ingredients_text.replace('&nbsp;', ' ').replace('&amp;', '&')
 
    
    # Extracting nutrition data as before
    table = soup.find('table', {'class': 'nutritionTable'})
    nutrition_data = []

    if table:
        header_row = table.find('tr')
        headers = [th.text.strip() for th in header_row.find_all('th')]

        # Add header row as a data point with placeholders
        if len(headers) >= 2:  # Ensure there are at least two headers (Nutrient and Per 100g)
            header_data = {
                'Title': 'Header Information',  # Placeholder
                'Description': '',
                'Price': '',
                'Ingredients': '',
                'Nutrient': headers[0],  # Should be 'Nutrient' or similar
                'Col1': headers[1],  # Header name for Per 100g values
                'Col2': headers[2] if len(headers) > 2 else 'Per Item',  # Header name for Per Item values, if it exists
                'Col3': headers[3] if len(headers) > 3 else '',  # Header name for Per Item values, if it exists
                'Col4': headers[4] if len(headers) > 4 else '',  # Header name for Per Item values, if it exists
                'url': url

                
            }
            nutrition_data.append(header_data)
      

        for row in table.find_all('tr')[1:]:
            columns = row.find_all('td')
            header = row.find('th')
            if header and len(columns) > 0:
                nutrient = header.text.strip()
                col1 = columns[0].text.strip() if len(columns) > 0 else "N/A"
                col2 = columns[1].text.strip() if len(columns) > 1 else "N/A"
                col3 = columns[2].text.strip() if len(columns) > 2 else "N/A"
                col4 = columns[3].text.strip() if len(columns) > 3 else "N/A"        
                data_point = {
                    'Title': title_text,
                    'Description': description_text,
                    'Price': price_text,
                    'Ingredients': ingredients_text,
                    'Nutrient': nutrient,
                    'Col1': col1,
                    'Col2': col2,
                    'Col3': col3,
                    'Col4': col4,
                    'url': url
                    
                }
                nutrition_data.append(data_point)

    all_nutrition_data.extend(nutrition_data)
    driver.close()

cake= pd.DataFrame(all_nutrition_data)
cake.tail(20)


,Title,Description,Price,Ingredients,Nutrient,Col1,Col2,Col3,Col4,url
2279,Warburtons Potato Cakes x6,Family bakers,£1.00,"WheatFlour [with Calcium, Iron, Niacin (B3) an...",Energy,1147kJ,516kJ,8400kJ,N/A,https://www.sainsburys.co.uk/gol-ui/product/wa...
2280,Warburtons Potato Cakes x6,Family bakers,£1.00,"WheatFlour [with Calcium, Iron, Niacin (B3) an...",Fat,10.4g,4.7g,70g,N/A,https://www.sainsburys.co.uk/gol-ui/product/wa...
2281,Warburtons Potato Cakes x6,Family bakers,£1.00,"WheatFlour [with Calcium, Iron, Niacin (B3) an...",of which saturates,3.8g,1.7g,20g,N/A,https://www.sainsburys.co.uk/gol-ui/product/wa...
2282,Warburtons Potato Cakes x6,Family bakers,£1.00,"WheatFlour [with Calcium, Iron, Niacin (B3) an...",Carbohydrate,38.7g,17.4g,260g,N/A,https://www.sainsburys.co.uk/gol-ui/product/wa...
2283,Warburtons Potato Cakes x6,Family bakers,£1.00,"WheatFlour [with Calcium, Iron, Niacin (B3) an...",of which sugars,6.1g,2.8g,90g,N/A,https://www.sainsburys.co.uk/gol-ui/product/wa...
2284,Warburtons Potato Cakes x6,Family bakers,£1.00,"WheatFlour [with Calcium, Iron, Niacin (B3) an...",Fibre,2.2g,1.0g,-,N/A,https://www.sainsburys.co.uk/gol-ui/product/wa...
2285,Warburtons Potato Cakes x6,Family bakers,£1.00,"WheatFlour [with Calcium, Iron, Niacin (B3) an...",Protein,5.1g,2.3g,50g,N/A,https://www.sainsburys.co.uk/gol-ui/product/wa...
2286,Warburtons Potato Cakes x6,Family bakers,£1.00,"WheatFlour [with Calcium, Iron, Niacin (B3) an...",Salt,1.80g,0.81g,6g,N/A,https://www.sainsburys.co.uk/gol-ui/product/wa...
2287,Header Information,,,,Typical Values,Per 100g,Per slice,% based on RI for Average Adult,,https://www.sainsburys.co.uk/gol-ui/product/sa...
2288,Sainsbury's Deliciously Free From Eric The Cat...,Chocolate sponge cake filled with chocolate fr...,£7.00,"INGREDIENTS:Sugar, Rapeseed Oil, Free RangeEgg...",Energy,1664kJ,932kJ,-,N/A,https://www.sainsburys.co.uk/gol-ui/product/sa...


In [5]:
cake.to_csv(r"C:\Users\wpw326\Desktop\Sainsbury's Cakes & tarts.csv", index=False)


nuts url